In [23]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.svm import LinearSVC, LinearSVR
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.datasets import load_boston, load_iris, load_diabetes, make_classification, make_regression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.metrics import r2_score
import numpy as np
import pandas as pd
from scipy.stats import spearmanr, pearsonr

import eli5
from eli5.sklearn import PermutationImportance

In [19]:
def dcg_score(y_true, y_score, k=10):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])
    gains = 2 ** y_true - 1
    # highest rank is 1 so +2 instead of +1
    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gains / discounts)

def ndcg_score(y_true, y_score, k=10):
    best = dcg_score(y_true, y_true, k)
    actual = dcg_score(y_true, y_score, k)
    return actual / best

In [46]:
def get_classification_datasets():
    res = []

    data = load_iris()
    res.append(('iris_binary', data.data, data.target != 0, data.feature_names))
    
    X, y = make_classification(n_informative=5, n_redundant=0)
    res.append(('CLF(n_informative=5, n_redundant=0)', X, y, None))
    
    X, y = make_classification(n_informative=5, n_redundant=4)
    res.append(('CLF(n_informative=5, n_redundant=4)', X, y, None))

    X, y = make_classification(n_informative=1, n_redundant=4, n_clusters_per_class=1)
    res.append(('CLF(n_informative=1, n_redundant=4)', X, y, None))

    X, y = make_classification(n_informative=20, n_redundant=0)
    res.append(('CLF(n_informative=20, n_redundant=0)', X, y, None))

    return res


def get_regression_datasets():
    res = []
    
    data = load_boston()
    res.append(('boston', data.data, data.target, data.feature_names))
    
    data = load_diabetes()
    res.append(('diabetese', data.data, data.target, None))
    
    X, y = make_regression(n_informative=5)
    res.append(('REG(n_informative=5)', X, y, None))
    
    X, y = make_regression(n_informative=5, effective_rank=2)
    res.append(('REG(n_informative=5, effective_rank=2)', X, y, None))

    X, y = make_regression(n_informative=1)
    res.append(('REG(n_informative=1)', X, y, None))

    X, y = make_regression(n_informative=20)
    res.append(('REG(n_informative=20)', X, y, None))

    return res


In [47]:
def get_classifiers():
    return [
        LogisticRegression(),
        LinearSVC(),
        RandomForestClassifier(),
        DecisionTreeClassifier(),
    ]


def get_regressors():
    return [
        make_pipeline(StandardScaler(), LinearRegression()),
        make_pipeline(StandardScaler(), LinearSVR()),
        RandomForestRegressor(),
        DecisionTreeRegressor(),
    ]
    

In [48]:
def get_explanations(est, X, y, feature_names):
    df_inspect = eli5.explain_weights_df(est, feature_names=feature_names, top=100)
    if isinstance(df_inspect.index, pd.MultiIndex):
        df_inspect.index = df_inspect.index.droplevel()
    df_inspect.index.name = None
    
    pi = PermutationImportance(est, cv='prefit', n_iter=10).fit(X, y)
    df_pi = eli5.explain_weights_df(pi, feature_names=feature_names, top=100)
    
    pi_cv = PermutationImportance(est, cv=5, n_iter=10).fit(X, y)
    df_picv = eli5.explain_weights_df(pi_cv, feature_names=feature_names, top=100)
        
    df = pd.concat([df_inspect.weight, df_pi.weight, df_picv.weight], axis=1)
    df.columns=['w_inspect', 'w_pi', 'w_picv']
    df = df.dropna() / df.abs().sum()
    return df

In [65]:
def get_scores(df):
    w_inspect_abs = df.w_inspect.abs().values
    def _scores(column):
        return {
            'SpearmanR': spearmanr(w_inspect_abs, column.values).correlation,
            'NDCG': ndcg_score(w_inspect_abs, column.values, 100000),
            'NDCG@5': ndcg_score(w_inspect_abs, column.values, 5),
            'Pearson': pearsonr(w_inspect_abs, column.values)[0],
#             'R^2': r2_score(w_inspect_abs, column.values),
            'L2': np.linalg.norm(w_inspect_abs - column.values),
        }
    return {
        'PI': _scores(df.w_pi),
        'PICV': _scores(df.w_picv),
    }

In [66]:
def get_name(est):
    if isinstance(est, Pipeline):
        est = est.steps[-1][1]
    return est.__class__.__name__

dfs = []
estimators = {}
scores = []

def _append(X, y, feature_names, dataset_name, est):
    est.fit(X, y)
    df = get_explanations(est, X, y, feature_names)
    name = get_name(est)
    estimators[name, dataset_name] = est
    dfs.append((name, dataset_name, df))
    for k, v in get_scores(df).items():
        scores.append((name, dataset_name, k, v))
    print("done: {}  {}".format(name, dataset_name))
    

for (dataset_name, X, y, feature_names) in get_classification_datasets():
    for clf in get_classifiers():
        _append(X, y, feature_names, dataset_name, clf)
        
for (dataset_name, X, y, feature_names) in get_regression_datasets():
    for reg in get_regressors():
        _append(X, y, feature_names, dataset_name, reg)

done: LogisticRegression  iris_binary
done: LinearSVC  iris_binary
done: RandomForestClassifier  iris_binary
done: DecisionTreeClassifier  iris_binary
done: LogisticRegression  CLF(n_informative=5, n_redundant=0)
done: LinearSVC  CLF(n_informative=5, n_redundant=0)
done: RandomForestClassifier  CLF(n_informative=5, n_redundant=0)
done: DecisionTreeClassifier  CLF(n_informative=5, n_redundant=0)
done: LogisticRegression  CLF(n_informative=5, n_redundant=4)
done: LinearSVC  CLF(n_informative=5, n_redundant=4)
done: RandomForestClassifier  CLF(n_informative=5, n_redundant=4)
done: DecisionTreeClassifier  CLF(n_informative=5, n_redundant=4)
done: LogisticRegression  CLF(n_informative=1, n_redundant=4)
done: LinearSVC  CLF(n_informative=1, n_redundant=4)
done: RandomForestClassifier  CLF(n_informative=1, n_redundant=4)
done: DecisionTreeClassifier  CLF(n_informative=1, n_redundant=4)
done: LogisticRegression  CLF(n_informative=20, n_redundant=0)
done: LinearSVC  CLF(n_informative=20, n_redu

In [67]:
df = pd.DataFrame([s[3] for s in scores])
df = df.assign(
    estimator=[s[0] for s in scores],
    dataset=[s[1] for s in scores],
    type=[s[2] for s in scores],
)
df

,L2,NDCG,NDCG@5,Pearson,SpearmanR,dataset,estimator,type
0,0.672614,0.984817,0.984817,0.833270,0.632456,iris_binary,LogisticRegression,PI
1,0.672541,0.984817,0.984817,0.833273,0.632456,iris_binary,LogisticRegression,PICV
2,0.675399,1.000000,1.000000,0.872404,0.948683,iris_binary,LinearSVC,PI
3,0.671430,0.999950,0.999950,0.873153,0.632456,iris_binary,LinearSVC,PICV
4,0.282843,0.964335,0.964335,0.968496,0.816497,iris_binary,RandomForestClassifier,PI
5,0.109003,1.000000,1.000000,0.993515,1.000000,iris_binary,RandomForestClassifier,PICV
6,0.000000,1.000000,1.000000,1.000000,1.000000,iris_binary,DecisionTreeClassifier,PI
7,0.834689,0.630930,0.630930,0.357553,0.272166,iris_binary,DecisionTreeClassifier,PICV
8,0.167495,0.971622,0.965187,0.872985,0.908544,"CLF(n_informative=5, n_redundant=0)",LogisticRegression,PI
9,0.255409,0.973934,0.981802,0.844037,0.820918,"CLF(n_informative=5, n_redundant=0)",LogisticRegression,PICV


In [68]:
df_pi = df[df.type=="PI"]
df_pi

,L2,NDCG,NDCG@5,Pearson,SpearmanR,dataset,estimator,type
0,0.672614,0.984817,0.984817,0.833270,0.632456,iris_binary,LogisticRegression,PI
2,0.675399,1.000000,1.000000,0.872404,0.948683,iris_binary,LinearSVC,PI
4,0.282843,0.964335,0.964335,0.968496,0.816497,iris_binary,RandomForestClassifier,PI
6,0.000000,1.000000,1.000000,1.000000,1.000000,iris_binary,DecisionTreeClassifier,PI
8,0.167495,0.971622,0.965187,0.872985,0.908544,"CLF(n_informative=5, n_redundant=0)",LogisticRegression,PI
10,0.123809,0.969013,0.932469,0.893776,0.934538,"CLF(n_informative=5, n_redundant=0)",LinearSVC,PI
12,0.322749,0.994469,0.989704,0.900801,0.885049,"CLF(n_informative=5, n_redundant=0)",RandomForestClassifier,PI
14,0.072285,0.993702,0.970213,0.981011,0.973103,"CLF(n_informative=5, n_redundant=0)",DecisionTreeClassifier,PI
16,0.273521,0.981474,0.973298,0.812503,0.817674,"CLF(n_informative=5, n_redundant=4)",LogisticRegression,PI
18,0.250067,0.966972,0.913251,0.768077,0.706813,"CLF(n_informative=5, n_redundant=4)",LinearSVC,PI


In [71]:
df_pi.groupby('estimator').mean()

,L2,NDCG,NDCG@5,Pearson,SpearmanR
estimator,,,,,
DecisionTreeClassifier,0.067172,0.997439,0.989606,0.983372,0.990131
DecisionTreeRegressor,0.100825,0.975222,0.963415,0.968772,0.979390
LinearRegression,0.268582,0.999978,0.999885,0.972810,0.997925
LinearSVC,0.327005,0.980478,0.957671,0.848220,0.803290
LinearSVR,0.341993,0.992524,0.989305,0.943220,0.896974
LogisticRegression,0.324789,0.981496,0.973374,0.862891,0.786834
RandomForestClassifier,0.319975,0.875767,0.765293,0.644987,0.659278
RandomForestRegressor,0.103122,0.996088,0.995512,0.988109,0.840017


In [72]:
df_pi.groupby('dataset').mean()

,L2,NDCG,NDCG@5,Pearson,SpearmanR
dataset,,,,,
"CLF(n_informative=1, n_redundant=4)",0.315954,0.868735,0.768673,0.641759,0.727879
"CLF(n_informative=20, n_redundant=0)",0.169107,0.971748,0.920533,0.819741,0.728945
"CLF(n_informative=5, n_redundant=0)",0.171585,0.982201,0.964393,0.912143,0.925309
"CLF(n_informative=5, n_redundant=4)",0.234315,0.984003,0.966543,0.882152,0.817877
REG(n_informative=1),0.274572,0.998082,0.995583,0.989532,0.877055
REG(n_informative=20),0.113707,0.979044,0.963679,0.944721,0.924267
REG(n_informative=5),0.161041,0.992570,0.990575,0.966880,0.878868
"REG(n_informative=5, effective_rank=2)",0.273500,0.998737,0.998977,0.978429,0.957326
boston,0.193210,0.999111,0.998143,0.968828,0.979396
